In [ ]:
import pandas as pd
import pyodbc
import pymssql
from pathlib import Path
from sqlalchemy import create_engine
import numpy as np
import urllib
import datetime as dt
import re

In [ ]:
# Read in base file
# This is where the link's at https://ccb.nv.gov/

In [ ]:
pd.options.display.min_rows = 50
pd.options.display.max_colwidth = 100

In [ ]:
# Change this to the date in the Excel file

update_date = dt.date(2021, 9, 23)

In [ ]:
# Don't know if I need to do this, but it makes it a little easier anyway

update_year = update_date.year
update_month = update_date.strftime("%m")
update_month_name = update_date.strftime("%B")

In [ ]:
nev_licenses = pd.read_excel(f'license_repository/{update_year}_{update_month}.xlsx',header=1)

In [ ]:
nev_licenses.head()

### SQL Connection

In [ ]:
# # Typing the driver directly into the create_engine kept not working, so I'm trying it this way
# driver="ODBC Driver 17 for SQL Server"
# engine = create_engine(f'mssql://LAPTOP-E6QKON1L/nv_cannabis?driver={driver}')
# engine_con = engine.connect()

In [ ]:
# This creates the connection to the Bespoke Engine's Oregon Cannabis files in SQL
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=nv_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine_con = engine.connect()

### Rename and Add Columns

In [ ]:
# This renames the columns that will be staying and ignores the others

nev_licenses.columns = ['license_number', 'name_legal', 'me_id', 'license_description']

In [ ]:
nev_licenses['is_current'] = 1

### License Types

In [ ]:
# # This creates a list of all of the different license types

# all_license_types = list(nev_licenses['License Type'].unique())

In [ ]:
# # This creates a function to create and fill the columns

# def create_license_type_col(lic_type):
#     col_name = lic_type.lower().replace(' ', '_')
#     nev_licenses[col_name] = ''
#     for n in range(len(nev_licenses)):
#         nev_licenses[col_name][n] = int(nev_licenses['License Type'][n] == lic_type)

In [ ]:
# # This runs the function for all of the license types

# for types in all_license_types:
#     create_license_type_col(types)

In [ ]:
# # Don't need this any more

# nev_licenses = nev_licenses.drop(columns=['License Type'])

### Name Legal Clean

In [ ]:
df_all = nev_licenses

In [ ]:
# df_all['name_legal_clean'] = df_all['name_legal']

# pd.set_option('mode.chained_assignment', None)

# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.lower() 
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, inc.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\binc.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\binc\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, llc.\b", "") 
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, llc\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, l.l.c.\b", "") 
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bllc\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bcorp.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bcorp\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bcorporation\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bco.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bco\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(",", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(".", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace("'", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.strip()

In [ ]:
df_all.head()

## Add Info From SQL

### Add the Old SQL Data

In [ ]:
old_nv_roll = pd.read_sql('nv_roll', engine)

In [ ]:
old_nv_contact = pd.read_sql('nv_contact', engine)

In [ ]:
old_nv_main = pd.read_sql('nv_main', engine)

In [ ]:
# # This is just to fix the roll up ids from df_first, and should remain commented out for all further changes

# old_roll_list = list(old_nv_roll['roll_up_id'])
# new_roll_list = []

# for roll in old_roll_list:
#     roll = str(roll.replace("NV-", ""))
#     while len(roll) < 5:
#         roll = "0" + str(roll)
#     roll = "NV-" + roll
#     new_roll_list.append(roll)

# old_nv_roll['roll_up_id'] = new_roll_list

### Merge on the Needed Information

In [ ]:
df_with_roll = df_all.merge(old_nv_roll, how='outer', on='license_number')

In [ ]:
df_with_con = df_with_roll.merge(old_nv_contact, how='outer', on='license_number')

In [ ]:
df_with_sql = df_with_con

In [ ]:
df_with_sql.head()

## Convert To and From CSV to Add Company Roll Up

In [ ]:
# Excel fucks this up if it's a number, so I'm adding text to keep it as a string and will remove it again later

for n in range(len(df_with_sql)):
    df_with_sql['license_number'][n] = str(df_with_sql['license_number'][n]) + '-LIC'

In [ ]:
df_with_sql.sort_values(by='name_legal', inplace=True)

In [ ]:
# We start by converting the Pandas Dataframe to a CSV file for manual additions

df_with_sql.to_csv(f'edited_files/df_{update_year}_{update_month}_to_edit.csv', index=False)

### Make Changes Before Continuing

In [ ]:
# This makes sure that the CSV was properly edited before reading it in

if pd.read_csv(f'edited_files/df_{update_year}_{update_month}_to_edit.csv')['company_roll_up'].isnull().sum() == 0:
    df_edited = pd.read_csv(f'edited_files/df_{update_year}_{update_month}_to_edit.csv')
else:
    print(error)

In [ ]:
# df_edited = pd.read_csv('license_repository/df_first.csv')

In [ ]:
df_edited.head()

In [ ]:
# This removes the -LIC from the license_number 

for n in range(len(df_edited)):
    df_edited['license_number'][n] = df_edited['license_number'][n].replace('-LIC', '')

In [ ]:
df_edited.head()

### Clean Contact

In [ ]:
df_contact_edit = df_edited

In [ ]:
# This cleans the contact_phone section

df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('tel','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('-','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('+','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('.','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('?','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('!','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace(':','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('(','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace(')','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('\n','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace(' ','')

In [ ]:
df_contact_edit

In [ ]:
# This will enable a little bit more cleaning of each

all_phones = list(df_contact_edit['contact_phone'])
all_emails = list(df_contact_edit['contact_email'])
all_websites = list(df_contact_edit['contact_website'])

In [ ]:
# This drops all of the phone numbers that are too short to be phone numbers

for x in range(len(all_phones)):
    if len(str(all_phones[x])) < 10:
        all_phones[x] = ''

In [ ]:
# This drops all of the phone numbers that are too long to be phone numbers

for x in range(len(all_phones)):
    if len(str(all_phones[x])) > 11:
        all_phones[x] = ''

In [ ]:
# This drops all of the instragram accounts and other non-email methods of contact

for x in range(len(all_emails)):
    if '@' not in str(all_emails[x]):
        all_emails[x] = ''

In [ ]:
# This drops all of the instragram accounts

for x in range(len(all_websites)):
    if 'insta' in str(all_websites[x]):
        all_websites[x] = ''

In [ ]:
# This drops all of the facebook accounts

for x in range(len(all_websites)):
    if 'facebook' in str(all_websites[x]):
        all_websites[x] = ''

In [ ]:
# And then we just add the edited lists back in to the dataframe

df_contact_edit['contact_phone'] = all_phones
df_contact_edit['contact_email'] = all_emails
df_contact_edit['contact_website'] = all_websites

In [ ]:
# Just filling the NaNs

df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].fillna('')
df_contact_edit['contact_email'] = df_contact_edit['contact_email'].fillna('')
df_contact_edit['contact_website'] = df_contact_edit['contact_website'].fillna('')

In [ ]:
df_contact_edit

### Seperate into Null and Fill

In [ ]:
# This creates two seperate dataframes for rows needing roll_up_ids and those that don't

df_edited_null = df_edited[df_edited['roll_up_id'].isnull()]
df_edited_fill = df_edited[df_edited['roll_up_id'].notnull()]

In [ ]:
# This creates all the roll_up_id numbers

new_comp = list(df_edited_null['company_roll_up'].unique())

current_roll_list = []
for roll in list(df_edited_fill['roll_up_id'].unique()):
    current_roll_list.append(int(str(roll)[3:]))

max_roll = max(current_roll_list)
# max_roll = 0 # Only for used the first time
    
new_roll = []
for n in range(len(new_comp)):
    max_roll += 1
    new_roll.append(max_roll)
    
new_nv_roll = []
for num in new_roll:
    num = str(num)
    while len(num) < 5:
        num = "0" + str(num)
    new_nv_roll.append(f'NV-{num}')

In [ ]:
df_edited_null

In [ ]:
df_edited_fill['roll_up_id']

In [ ]:
new_nv_roll

In [ ]:
new_comp

In [ ]:
# This uses the two lists created above to add roll_up_ids to the matching company_roll_ups to the df_edited

df_edited_null = df_edited_null.reset_index().drop(columns={'index'})

for n in range(len(df_edited_null)):
    df_edited_null['roll_up_id'][n] = new_nv_roll[new_comp.index(df_edited_null['company_roll_up'][n])]

In [ ]:
df_edited_null.head()

In [ ]:
# This reattaches the two dataframes now that the roll_up_ids have been filled

df_populated = df_edited_fill.append(df_edited_null)

### Clean for SQL

In [ ]:
# This creates the list of the licenses in the dataframe and drops thse already in SQL 
licenses_for_sql = []
for lic in list(df_populated['license_number']):
    if lic not in list(old_nv_roll['license_number']):
        licenses_for_sql.append(lic)

In [ ]:
# This creates a dataframe of the new licenses and merges in the df_populated info for those new licenses
df_for_sql = pd.DataFrame(licenses_for_sql, columns={'license_number'})
df_for_sql = df_for_sql.merge(df_populated, how='left')

In [ ]:
df_for_sql.columns

### Seperate Data for SQL

In [ ]:
# This just seperates the dataframe into the main and roll information that will be put into SQL

df_nv_main = df_for_sql[['license_number', 'name_legal', 'license_description', 'me_id', 'is_current']]

df_nv_roll = df_for_sql[['license_number', 'roll_up_id', 'company_roll_up']]

df_nv_contact = df_for_sql[['license_number', 'contact_email', 'contact_phone', 'contact_website']]

In [ ]:
# This adds he date issued for the newly added licenses
df_nv_main['date_uploaded'] = update_date
df_nv_main['date_uploaded'] = pd.to_datetime(df_nv_main['date_uploaded'])
df_nv_main

In [ ]:
print(stop)

### To SQL

In [ ]:
df_nv_roll.to_sql('nv_roll', con=engine_con, if_exists='append', index=False)

In [ ]:
df_nv_main.to_sql('nv_main', con=engine_con, if_exists='append', index=False)

In [ ]:
df_nv_contact.to_sql('nv_contact', con=engine_con, if_exists='append', index=False)

### Main Alter

In [ ]:
df_for_alter = df_contact_edit[['license_number', 'name_legal', 'license_description', 'me_id', 'is_current']]

In [ ]:
df_for_alter

In [ ]:
# This marks out the non-current licenses for alteration

for n in range(len(df_for_alter)):
    if df_for_alter['license_number'][n] in list(df_for_alter[df_for_alter['name_legal'].isnull()]['license_number']):
        df_for_alter['is_current'][n] = 0

In [ ]:
# This isn't technically needed, but my OCD was acting up
df_for_alter['is_current'] = df_for_alter['is_current'].astype(int)

In [ ]:
# This creates the dataframe for adding old info to the non current licenses, so that only the 
df_not_current = pd.DataFrame(list(df_for_alter[df_for_alter['is_current'] == 0]['license_number']), columns=['license_number']).merge(old_nv_main)
df_not_current['is_current'] = 0

In [ ]:
# There's probably a cleaner way to do this, but it's done
df_for_alter = df_for_alter.append(df_not_current).reset_index().drop(columns={'index', 'date_uploaded'})
df_for_alter = df_for_alter[df_for_alter['name_legal'].notnull()]

In [ ]:
# These are all the licenses that will need altering
for_alter = []
for lic in list(df_for_alter['license_number']):
#     print(old_nv_main[old_nv_main['license_number'] == lic]['name_legal'].values, df_for_alter[df_for_alter['license_number'] == lic]['name_legal'].values)
    if str(df_for_alter[df_for_alter['license_number'] == lic]['name_legal'].values) != str(old_nv_main[old_nv_main['license_number'] == lic]['name_legal'].values):
        for_alter.append(lic)
    if str(df_for_alter[df_for_alter['license_number'] == lic]['is_current'].values) != str(old_nv_main[old_nv_main['license_number'] == lic]['is_current'].values):
        for_alter.append(lic)
    if str(df_for_alter[df_for_alter['me_id'] == lic]['is_current'].values) != str(old_nv_main[old_nv_main['me_id'] == lic]['is_current'].values):
        for_alter.append(lic)
    if str(df_for_alter[df_for_alter['license_description'] == lic]['is_current'].values) != str(old_nv_main[old_nv_main['license_description'] == lic]['is_current'].values):
        for_alter.append(lic)
for_alter = list(set(for_alter))

In [ ]:
# This creates the dataframe to send to SQL for altering
df_sql_alter = pd.DataFrame(for_alter, columns=['license_number'])
df_sql_alter = df_sql_alter.merge(df_for_alter, on='license_number')

In [ ]:
df_sql_alter.to_sql('nv_main_alter', con=engine_con, if_exists='replace', index=False)